<a href="https://colab.research.google.com/github/zavaleta/indices_climaticos/blob/master/NetCDF4_python2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NetCDF4